In [265]:
import vk
ses = vk.Session(access_token=token)  # Авторизация
vk_api = vk.API(ses)

In [213]:
group = vk_api.groups.getMembers(group_id='v5inf',v=5.92)

In [214]:
def getUserId(link):
    id = link
    if 'vk.com/' in link: #  проверяем эту ссылку
        id = link.split('/')[-1]  # если да, то получаем его последнюю часть
    if not id.replace('id', '').isdigit(): # если в нем после отсечения 'id' сами цифры - это и есть id 
        id = vk_api.utils.resolveScreenName(screen_name=id,v=5.92)['object_id'] # если нет, получаем id с помощью метода API
    else:
        id = id.replace('id', '')
    return int(id)

In [215]:
# count это количество запросов (и количество постов = 100 * count постов)
def getLikes(user_id, cnt, vk_api):
    import time
    # подписки пользователя
    subscriptions_list = vk_api.users.getSubscriptions(user_id=user_id,extended=0,v=5.92)['groups']['items']
    # формируем список id, который нужно передать в следующий метод
    groups_list = ['-' + str(x) for x in subscriptions_list]
    posts = {}
    # формируем ленту новостей
    newsfeed = vk_api.newsfeed.get(
        filters='post',
        source_ids=', '.join(groups_list),
        count=100, timeout=10,v=5.92)
    # добавляем посты в словарь в формате id_поста: id_группы
    posts.update({x['post_id']: x['source_id'] for x in newsfeed['items']})
    # нужно для получения следующей партии
    # если требуется более одного запроса — делаем остаток в цикле
    if cnt != 1:
        for c in range(cnt - 1):
            next_from = newsfeed['next_from']
            kwargs = {
                'start_from': next_from,
                'filters': 'post',
                'source_ids': ', '.join(groups_list),
                'count': 100,
                'timeout': 10,
                'v':5.92
            }
            newsfeed = vk_api.newsfeed.get(**kwargs)

            posts.update({x['post_id']: x['source_id'] for x in newsfeed['items']})
            time.sleep(1)
    liked_posts = []

    print('Лайкнутые посты:')
    for post in posts.items():
        
        try:
            itemID = post[0]
            ownerID = post[1]
            timeOut = 5
            isLiked = vk_api.likes.isLiked(
                user_id=user_id,
                item_id=itemID,
                type='post',
                owner_id=ownerID,
                timeout=timeOut)
            print(isLiked)
        except Exception:
#             print('ERROR! ' + 'vk.com/wall{0}_{1}'.format(post[1], post[0]))
            isLiked = 0

        if isLiked:
            liked_posts.append('vk.com/wall{0}_{1}'.format(post[1], post[0]))
            print('vk.com/wall{}_{}'.format(post[1], post[0]))
            time.sleep(1)
    return liked_posts

In [216]:
user_id = 
user_id = getUserId(user_id)
#getLikes(user_id, 5, vk_api) # Сканируем 5 * 100 = 500 постов

In [576]:
import json
import time
columns = []
name = []
female = []
genres = []
def getFriends(user_id,vk_api):
    # друзья пользователя
    friends_list = vk_api.friends.get(user_id=user_id,v=5.92,fields='nickname')['items']
    data_user = []
    count = 0
    for user in friends_list: 
        if count == 50:
            break
        data = []
        data_genre = []
        if 'deactivated' in user:
            continue
        user_info = vk_api.users.getSubscriptions(user_id=user['id'],fields='activity',extended=1,count=5,v=5.102)['items']
        for x in user_info:
            if x['type'] == 'profile':
                continue
            data.append(x['name'])
            data_genre.append(x['activity'])
        if data:
            data_user.append(data)
            genres.append(data_genre)
            columns.append(user['id'])
            name.append(user['first_name'])
            female.append(user['last_name'])
        #data_user.update({user['id']:data})
        time.sleep(0.3)
        count += 1
    return data_user

In [577]:
data = getFriends(user_id,vk_api)